# Measuring uncertainties in the RMSE metric

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

import sys
sys.path.append('../../src')
from multiclass import *

import torch 

from contextlib import redirect_stdout
from IPython.display import display

PATH = os.path.abspath('../..')

# k-fold train-test split
Since we're not optimizing any hyperparameters, we can simply look at how the validation (= test) scores change using a $k$-fold split.

In [6]:
# get SDSS object IDs of all images
obj_ids = np.array([os.path.splitext(os.path.basename(fname))[0] for fname in glob(f'{PATH}/images/*.jpg')]).astype(np.int64)

In [102]:
df_full = pd.read_csv(f'{PATH}/catalogs/SDSSspecgalsDR14_boada.csv', index_col=0)

# get only those with images and save out
df_images = df_full.loc[df_full.index.intersection(obj_ids)]

# drop duplicates
df_images = df_images.loc[~df_images.index.duplicated()]
df_images.to_csv(f'{PATH}/catalogs/all-images.csv', columns=df_images.columns)

## Predict only [O/H]

In [104]:
# save out only metallicity
df = df_images['oh_p50']
train_label_csv = f'{PATH}/catalogs/all-images-oh_p50.csv'
df.to_csv(train_label_csv, header=True)

In [107]:
def get_data(arch, sz, bs, val_idxs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.1)
    return ImageClassifierData.from_multiclass_csv(PATH, 'images', train_label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name=None, num_workers=16)


In [108]:
# params
sz = 128
bs = 128
arch = resnet34

# Train and predict on the full data set

In [34]:
np.random.seed(1234)
n = len(df)
all_idxs = np.random.permutation(np.arange(n))

df = pd.read_csv(train_label_csv, index_col=0)

for k in range(5):

    val_idxs = all_idxs[int(k*n/5):int((k+1)*n/5)]
    
    data = get_data(arch, sz, bs, val_idxs)

    learn = ConvLearner.pretrained(arch, data, pretrained=True)
    learn.crit = rmse
    
    # training regiment
    lr = 1e-1
    learn.fit(lr, 2)

    lrs = [1e-2, 3e-2, 1e-1]
    learn.unfreeze()

    learn.fit(lrs, 1)
    learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)
    
    # save out the learner
    learn.save(f'all-images-oh_p50-fold_{k}')

epoch      trn_loss   val_loss                                 
    0      0.154848   0.153678  
    1      0.16004    0.138506                                 


epoch      trn_loss   val_loss                                 
    0      0.123661   0.116784  


epoch      trn_loss   val_loss                                  
    0      0.092422   0.087587  
    1      0.102603   0.08817                                  
    2      0.093135   0.086709                                  
    3      0.108331   0.138349                                 
    4      0.106008   0.089385                                 
    5      0.095354   0.08388                                   
    6      0.088996   0.084193                                  


epoch      trn_loss   val_loss                                 
    0      0.157462   0.139975  
    1      0.157764   0.134189                                 


epoch      trn_loss   val_loss                                 
    0      0.118175   0.102226  


...

epoch      trn_loss   val_loss                                 
    0      0.1282     0.208105  


epoch      trn_loss   val_loss                                  
    0      0.093942   0.086627  
    1      0.105392   0.095397                                 
    2      0.092877   0.084936                                  
    3      0.106841   0.092219                                 
    4      0.10705    0.087114                                 
    5      0.096016   0.084959                                  
    6      0.088405   0.083041                                  


epoch      trn_loss   val_loss                                 
    0      0.162266   0.193164  
    1      0.154895   0.137904                                 


epoch      trn_loss   val_loss                                 
    0      0.121469   0.131841  


epoch      trn_loss   val_loss                                  
    0      0.091164   0.08671   
    1      0.102608   0.093794                                 
    2      0.091263   0.085797                                  
    3      0.113851   0.092231                                 
    4      0.104029   0.118459                                 
    5      0.093771   0.086587                                  
    6      0.086569   0.084086                                  


In [42]:
display(f'Train-validation split: {np.round(4*n/5).astype(int)} {np.round(n/5).astype(int)}')

'Train-validation split: 93143 23286'

## Without TTA

In [65]:
metrics = np.array([0.0842, 0.0828, 0.0846, 0.0830, 0.0841])
display(f'For this (slightly larger) data set, and without TTA, we find that the RMSE is about {metrics.mean():.4f} +/- {metrics.std():.4f}.')

'For this (slightly larger) data set, and without TTA, we find that the RMSE is about 0.0837 +/- 0.0007.'

## Use TTA

In [62]:
for k in tqdm_notebook(range(5)):

    val_idxs = all_idxs[int(k*n/5):int((k+1)*n/5)]
    
    data = get_data(arch, sz, bs, val_idxs)

    learn = ConvLearner.pretrained(arch, data, pretrained=True)
    learn.crit = rmse
    
    learn.load(f'all-images-oh_p50-fold_{k}')
    
    Z_pred, Z_true = learn.TTA()
    display(f'For fold {k}, the RMSE = {rmse_np(Z_pred.mean(axis=0).flatten(), Z_true.flatten()):.4f}')

'For fold 0, the RMSE = 0.0840'

'For fold 1, the RMSE = 0.0830'

'For fold 2, the RMSE = 0.0844'

'For fold 3, the RMSE = 0.0831'

'For fold 4, the RMSE = 0.0837'

In [64]:
metrics = np.array([0.0840, 0.0830, 0.0844, 0.0831, 0.0837])
display(f'For this (slightly larger) data set, and with TTA, we find that the RMSE is about {metrics.mean():.4f} +/- {metrics.std():.4f}.')

'For this (slightly larger) data set, and with TTA, we find that the RMSE is about 0.0836 +/- 0.0005.'

# Nested cross-validation

In [109]:
train_label_csv = f'{PATH}/catalogs/all-images-oh_p50.csv'
df = pd.read_csv(train_label_csv, index_col=0)

In [110]:
def get_data(arch, sz, bs, val_idxs, test_dir):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_top_down, max_zoom=1.1)
    return ImageClassifierData.from_multiclass_csv(PATH, 'images', train_label_csv, tfms=tfms,
                    suffix='.jpg', val_idxs=val_idxs, test_name=test_dir, num_workers=16)

In [111]:
sz = 128
bs = 128
arch = resnet34

## Training loop

In [ ]:
n = len(df)
df = pd.read_csv(train_label_csv, index_col=0)

# shuffle
df = df.sample(n, random_state=1234)
all_idxs = np.arange(n)

results_file = f'{PATH}/results/nested-crossval-results.txt'

with open(results_file, 'a') as f:
    
    print(f'Total train + val + test size = {n}', file=f, flush=True)

    # test split
    for k in range(5):
       
        print(f'  Test fold {k+1}/5 (size = {int((k+1)*n/5)-int(k*n/5)})', file=f, flush=True)
        test_idxs = all_idxs[int(k*n/5):int((k+1)*n/5)]
        
        df_test = df.iloc[test_idxs]

        # copy images over to test dir
        test_dir = f'nested_csv-test-fold_{k}'
        
        print('Copying test images')
        for objid in tqdm(df_test.index, total=len(df_test.index)):
            shutil.copyfile(f'{PATH}/images/{objid}.jpg', f'{PATH}/{test_dir}/{objid}.jpg')
        
        train_idxs = np.array(list(set(all_idxs) - set(test_idxs)))
        df_train = df.iloc[train_idxs]

        n_train = len(df_train)

        # crossval split
        for k_cv in range(4):

            val_idxs = train_idxs[int(k_cv*n_train/5):int((k_cv+1)*n_train/5)]
            data = get_data(arch, sz, bs, val_idxs, test_dir)

            learn = ConvLearner.pretrained(arch, data, pretrained=True)
            learn.crit = rmse

            # training regiment
            lr = 1e-1
            learn.fit(lr, 2)

            lrs = [1e-2, 3e-2, 1e-1]
            learn.unfreeze()

            learn.fit(lrs, 1)
            learn.fit(lrs, 3, cycle_len=1, cycle_mult=2)

            # save out the learner
            learn.save(f'nested_crossval-oh_p50-test_{k}-cv_{k_cv}')
            
            # report cross-validation results
            print(f'    {learn.sched.val_losses[-1][0]:.5f} (Validation) fold {k_cv+1}/4, size = {int((k_cv+1)*n_train/5) - int(k_cv*n_train/5)}', file=f, flush=True)

            # do test with TTA
            preds, _ = learn.TTA(is_test=True)
            Z_pred = preds.mean(axis=0).flatten()
            print(len(Z_pred))

            test_ids = np.array([os.path.split(fname)[1][:-4] for fname in learn.data.test_ds.fnames], dtype=np.int64)
            Z_test = df.loc[test_ids].values.flatten()
            print(len(Z_test))
            print(f'    {rmse_np(Z_pred, Z_test):.5f} (Test)', file=f, flush=True)



Copying test images
100%|██████████| 23285/23285 [00:01<00:00, 13514.03it/s]


epoch      trn_loss   val_loss                                 
    0      0.154945   0.140887  
 33%|███▎      | 506/1529 [00:48<01:37, 10.53it/s, loss=0.159]

## Parse results

In [13]:
results_file = f'{PATH}/results/nested-crossval-results.txt'

with open(results_file, 'r') as f:
    lines = f.readlines()
    res = np.zeros((2, 4, 5))
    
    idx = 0
    for line in lines:
        if line[:4] == '    ':
            indices = np.unravel_index(idx, res.shape, order='F')
            idx += 1
            res[indices] = float(line[4:11])

In [14]:
res

array([[[0.08519, 0.08576, 0.08371, 0.08405, 0.0836 ],
        [0.08292, 0.08464, 0.08447, 0.08311, 0.08342],
        [0.08592, 0.08832, 0.08698, 0.08432, 0.08346],
        [0.08395, 0.08511, 0.08376, 0.08583, 0.08451]],

       [[0.08381, 0.08425, 0.0815 , 0.08213, 0.08216],
        [0.08271, 0.08382, 0.08326, 0.08171, 0.08187],
        [0.08369, 0.08851, 0.0849 , 0.08201, 0.08234],
        [0.08276, 0.08373, 0.08235, 0.08341, 0.08175]]])

In [21]:
val, test = res

- 1st axis is `[val, test]`
- 2nd axis is `[val1, val2, val3, val4]`
- 3rd axis is `[test1, test2, test3, test4, test5]`

### Use best validation score

In [57]:
best_idxs = val.argmin(axis=0)

best_scores = np.zeros((5,))
for j, t in enumerate(test.T):
    best_scores[j] = t[best_idxs[j]]

print(best_scores)

print(f'Best scores = {best_scores.mean():.4f} +/- {best_scores.std():.4f}')

[0.08271 0.08382 0.0815  0.08171 0.08187]
Best scores = 0.0823 +/- 0.0009


### Use weighted validation score

In [58]:
val_weights = val**-2 / np.sum(val**-2, axis=0)
print(val_weights)

[[0.2458  0.25095 0.25594 0.25155 0.25086]
 [0.25944 0.25764 0.25136 0.25728 0.25195]
 [0.24164 0.23661 0.23706 0.24994 0.2517 ]
 [0.25312 0.2548  0.25564 0.24123 0.24549]]


In [59]:
weighted_scores = (test * val_weights).sum(axis=0)
print(weighted_scores)
print(f'Weighted scores = {weighted_scores.mean():.4f} +/- {weighted_scores.std():.4f}')

[0.08323 0.08501 0.08297 0.0823  0.08203]
Weighted scores = 0.0831 +/- 0.0010


### Use average scores

In [60]:
avg_scores = test.mean(axis=0)
print(avg_scores)
print(f'Average scores = {avg_scores.mean():.4f} +/- {avg_scores.std():.4f}')

[0.08324 0.08508 0.083   0.08231 0.08203]
Average scores = 0.0831 +/- 0.0011


### Use weighted validation score (with a minimum score of 0.080)

In [63]:
min_val = 0.080
reval = (val - min_val)

reval_weighted = reval**-2 / np.sum(reval**-2, axis=0)
print(reval_weighted)

[[0.15028 0.23305 0.33956 0.24647 0.26125]
 [0.47476 0.35914 0.23391 0.41797 0.28947]
 [0.1155  0.1117  0.09593 0.21662 0.28282]
 [0.25945 0.29611 0.33059 0.11894 0.16646]]


In [64]:
reval_scores = (test * reval_weighted).sum(axis=0)
print(reval_scores)
print(f'Validation re-weighted scores = {reval_scores.mean():.4f} +/- {reval_scores.std():.4f}')

[0.083   0.08442 0.08252 0.08208 0.08206]
Validation re-weighted scores = 0.0828 +/- 0.0009


### Results
Using the best validation score gives the best result, unsurprisingly. The scores are actually quite low and drop down to $0.0815$ dex. Overall the 5-fold crossval results give us $0.0823 \pm 0.0009$, showing that the scatter is indeed low.